In [1]:
%pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.6/26.6 MB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.2/38.2 MB 15.3 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.16.1
    Uninstalling scipy-1.16.1:
      Successfully uninstalled scipy-1.16.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatibl

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from gensim.models.ldamodel import LdaModel

# Load the LDA model
loaded_lda_model = LdaModel.load('/content/drive/MyDrive/Colab Notebooks/alex/models/lda_20_topics/lda_model_20_topics.gensim')

# Now you can access information from the loaded model.
# For example, you can print the topics:
print("Topics:")
for topic_id, topic in loaded_lda_model.print_topics():
    print(f"Topic {topic_id}: {topic}")

# You can also access the dictionary (id2word)
print("\nDictionary:")
print(loaded_lda_model.id2word)

# And the number of topics
print("\nNumber of topics:")
print(loaded_lda_model.num_topics)

Topics:
Topic 0: 0.058*"coach" + 0.029*"basketball" + 0.014*"year" + 0.013*"coaching" + 0.013*"head" + 0.010*"program" + 0.010*"assistant" + 0.010*"season" + 0.009*"championship" + 0.009*"college"
Topic 1: 0.028*"school" + 0.022*"year" + 0.021*"player" + 0.020*"high" + 0.020*"season" + 0.018*"team" + 0.018*"college" + 0.013*"coach" + 0.012*"said" + 0.011*"transfer"
Topic 2: 0.036*"gopher" + 0.026*"hockey" + 0.025*"minnesota" + 0.020*"game" + 0.016*"goal" + 0.014*"team" + 0.013*"season" + 0.009*"state" + 0.009*"first" + 0.008*"four"
Topic 3: 0.007*"nil" + 0.007*"people" + 0.005*"day" + 0.005*"one" + 0.005*"say" + 0.005*"family" + 0.005*"trump" + 0.004*"life" + 0.004*"new" + 0.004*"social"
Topic 4: 0.016*"ncaa" + 0.016*"said" + 0.015*"college" + 0.015*"school" + 0.014*"athlete" + 0.012*"sport" + 0.011*"state" + 0.009*"university" + 0.008*"football" + 0.008*"would"
Topic 5: 0.024*"woman" + 0.018*"u" + 0.015*"championship" + 0.015*"olympic" + 0.015*"world" + 0.012*"team" + 0.010*"first" + 

In [3]:
from gensim import corpora

# Load the corpus from the .mm file
corpus = corpora.MmCorpus('/content/drive/MyDrive/Colab Notebooks/alex/models/dictionary_and_corpus/corpus.mm')

print("Corpus loaded successfully.")
# You can optionally print the corpus to inspect its structure
# print(list(corpus))

Corpus loaded successfully.


In [4]:
import pandas as pd

# Load the tokenized data from the Excel file
try:
    tokenized_data_df = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/alex/updated_ tokenized_newspaper_data.xlsx')
    print("Excel file loaded successfully.")
    # Assuming the original text is in a column named 'text' or similar.
    # Please adjust the column name if it's different in your Excel file.
    if 'Main_text_tokenized' in tokenized_data_df.columns:
        texts = tokenized_data_df['Main_text_tokenized'].tolist()
        print(f"Extracted {len(texts)} documents.")
    else:
        texts = None
        print("Could not find a 'text' column in the Excel file. Please make sure the column containing original texts is named 'text' or update the code.")

except FileNotFoundError:
    print("Error: The file was not found at the specified path.")
except Exception as e:
    print(f"An error occurred while reading the Excel file: {e}")

Excel file loaded successfully.
Extracted 149703 documents.


In [5]:
#TFinding the dominant topic in each sentence

def format_topics_sentences(ldamodel=loaded_lda_model, corpus=corpus, texts=texts): # Updated to use loaded_lda_model and corpus, and texts=None
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = pd.concat([sent_topics_df, pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]).to_frame().T], ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

     # Add original text to the end of the output
    if texts is not None: # Added check if texts is not None
        contents = pd.Series(texts)
        sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


# Assuming texts is available as data_words or you provide another list/Series of texts
# df_topic_sents_keywords = format_topics_sentences(ldamodel=loaded_lda_model, corpus=corpus, texts=texts) # Updated to use loaded_lda_model and corpus, commented out as data_words is undefined
# To run this cell, define `data_words` with your text data before calling the function.
# For example:
# data_words = ["your document text 1", "your document text 2", ...]
# Then uncomment the line below:
df_topic_sents_keywords = format_topics_sentences(ldamodel=loaded_lda_model, corpus=corpus, texts=texts)

# Format
if 'df_topic_sents_keywords' in locals(): # Added check if df_topic_sents_keywords is defined
    df_dominant_topic = df_topic_sents_keywords.reset_index()
    df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']
    # Show
    display(df_dominant_topic.head(20)) # Use display for better formatting

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,10,0.3776,"said, like, would, going, get, year, one, team...","['14', 'year', 'coaching', '39', 'year', 'livi..."
1,1,0,0.2295,"coach, basketball, year, coaching, head, progr...","['see', 'sidebar', '02c', 'ruston', 'la', 'din..."
2,2,10,0.3691,"said, like, would, going, get, year, one, team...","['gave', 'garden', 'exactly', 'missed', 'lonel..."
3,3,10,0.2876,"said, like, would, going, get, year, one, team...","['indianapolis', 'ten', 'year', 'ago', 'chuck'..."
4,4,12,0.3326,"draft, season, nba, player, pick, league, team...","['robinson', 'talk', 'ranger', 'stanley', 'cup..."
5,5,7,0.2607,"per, point, 2024, season, ppg, guard, g, last,...","['usc', 'conference', 'pacific10', 'coach', 'm..."
6,6,12,0.4864,"draft, season, nba, player, pick, league, team...","['ranger', 'signed', 'center', 'michael', 'nyl..."
7,7,12,0.37,"draft, season, nba, player, pick, league, team...","['gavitt', 'five', 'others', 'elected', 'hall'..."
8,8,10,0.2839,"said, like, would, going, get, year, one, team...","['almost', 'mythical', 'element', 'lsu', 'seni..."
9,9,3,0.2839,"nil, people, day, one, say, family, trump, lif...","['ncaa', 'woman', 'jim', 'izard', 'diana', 'vi..."


In [6]:
# Display the first few rows of the dataframe to see the structure
display(df_dominant_topic.head())

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,10,0.3776,"said, like, would, going, get, year, one, team...","['14', 'year', 'coaching', '39', 'year', 'livi..."
1,1,0,0.2295,"coach, basketball, year, coaching, head, progr...","['see', 'sidebar', '02c', 'ruston', 'la', 'din..."
2,2,10,0.3691,"said, like, would, going, get, year, one, team...","['gave', 'garden', 'exactly', 'missed', 'lonel..."
3,3,10,0.2876,"said, like, would, going, get, year, one, team...","['indianapolis', 'ten', 'year', 'ago', 'chuck'..."
4,4,12,0.3326,"draft, season, nba, player, pick, league, team...","['robinson', 'talk', 'ranger', 'stanley', 'cup..."


In [7]:
# Sort documents within each topic by percentage contribution in descending order
sorted_documents_by_topic = df_dominant_topic.sort_values(by=['Dominant_Topic', 'Topic_Perc_Contrib'], ascending=[True, False])

# Display the sorted document indexes for each topic
print("Document Indexes for Each Topic (Sorted by Percentage Contribution):")
for topic_num in sorted_documents_by_topic['Dominant_Topic'].unique():
    print(f"\nTopic {topic_num}:")
    # Get documents for the current topic and select only the 'Document_No' column
    topic_documents = sorted_documents_by_topic[sorted_documents_by_topic['Dominant_Topic'] == topic_num]['Document_No']
    # Display the document indexes (first 20 as an example)
    print(topic_documents.tolist()[:20])

Document Indexes for Each Topic (Sorted by Percentage Contribution):

Topic 0:
[77310, 40770, 136344, 70117, 127111, 133169, 59899, 21185, 147967, 18434, 43756, 129285, 43203, 39241, 23956, 61747, 42625, 131189, 56693, 27227]

Topic 1:
[55253, 23432, 66229, 3559, 79820, 70245, 15491, 26292, 51516, 3514, 4289, 63502, 13649, 63873, 72095, 83855, 72451, 29071, 34546, 62535]

Topic 2:
[2563, 83747, 86239, 71938, 137523, 55680, 137336, 5021, 142444, 139189, 99549, 137533, 109804, 82349, 7982, 21692, 55681, 45903, 112427, 121171]

Topic 3:
[30262, 72564, 29750, 106623, 89946, 80583, 60071, 18918, 95222, 26015, 6303, 119581, 43917, 38776, 55847, 6724, 55902, 104746, 40564, 147347]

Topic 4:
[70792, 32177, 70785, 127699, 138582, 112461, 33945, 43676, 89536, 143166, 143165, 105593, 43289, 24192, 101598, 133827, 47889, 129820, 131623, 132902]

Topic 5:
[21267, 24696, 31532, 27131, 1928, 24580, 3776, 3777, 103299, 35645, 48405, 143889, 13955, 122244, 27528, 54726, 144713, 59435, 131635, 83131]

T

In [8]:
# Get the total number of documents for each dominant topic
topic_counts = df_dominant_topic['Dominant_Topic'].value_counts().sort_index()

print("Total number of documents for each topic:")
display(topic_counts)

Total number of documents for each topic:


,count
Dominant_Topic,
0,4575
1,7568
2,3923
3,7320
4,14313
5,6935
6,16703
7,1870
8,12255


In [9]:
# Get the top 100 documents for each topic
top_100_documents_per_topic = pd.DataFrame()

for topic_num in sorted_documents_by_topic['Dominant_Topic'].unique():
    # Get the top 100 documents for the current topic
    top_100 = sorted_documents_by_topic[sorted_documents_by_topic['Dominant_Topic'] == topic_num].head(100)
    # Append to the results DataFrame
    top_100_documents_per_topic = pd.concat([top_100_documents_per_topic, top_100], ignore_index=True)

# Display the top 100 documents for each topic
print("Top 100 Documents for Each Topic (Sorted by Percentage Contribution):")

# Iterate through topics and display the top documents for each
for topic_num in top_100_documents_per_topic['Dominant_Topic'].unique():
    print(f"\n--- Topic {topic_num} ---")
    display(top_100_documents_per_topic[top_100_documents_per_topic['Dominant_Topic'] == topic_num])

Top 100 Documents for Each Topic (Sorted by Percentage Contribution):

--- Topic 0 ---


,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,77310,0,0.8944,"coach, basketball, year, coaching, head, progr...","['denver', 'watching', 'ncaa', 'tournament', '..."
1,40770,0,0.8565,"coach, basketball, year, coaching, head, progr...","['ninetime', 'winner', 'michelle', 'kwan', 'ye..."
2,136344,0,0.8416,"coach, basketball, year, coaching, head, progr...","['august', '2004', 'michael', 'oher', 'living'..."
3,70117,0,0.7034,"coach, basketball, year, coaching, head, progr...","['nhl', 'reportedly', 'signed', 'ilya', 'koval..."
4,127111,0,0.6946,"coach, basketball, year, coaching, head, progr...","['tournament', 'played', 'indiana', 'mostly', ..."
...,...,...,...,...,...
95,90200,0,0.5406,"coach, basketball, year, coaching, head, progr...","['jerry', 'bus', 'enroll', 'usc', 'graduate', ..."
96,114647,0,0.5385,"coach, basketball, year, coaching, head, progr...","['back', 'snark', 'age', 'mlb', 'player', 'ass..."
97,133395,0,0.5382,"coach, basketball, year, coaching, head, progr...","['popular', 'refrain', 'talking', 'kyle', 'nep..."
98,85950,0,0.5371,"coach, basketball, year, coaching, head, progr...","['note', 'college', 'basketball', 'preview', '..."



--- Topic 1 ---


,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
100,55253,1,0.8532,"school, year, player, high, season, team, coll...","['nearly', 'two', 'year', 'since', 'ncaa', 'op..."
101,23432,1,0.8312,"school, year, player, high, season, team, coll...","['upset', 'stomach', 'headache', 'dealt', 'day..."
102,66229,1,0.8183,"school, year, player, high, season, team, coll...","['pasadena', 'ucla', 'deserves', 'play', 'bowl..."
103,3559,1,0.7841,"school, year, player, high, season, team, coll...","['sport', 'monday', 'round', 'one', 'usc', 'li..."
104,79820,1,0.7787,"school, year, player, high, season, team, coll...","['sport', 'delicious', 'duality', 'definite', ..."
...,...,...,...,...,...
195,51017,1,0.5859,"school, year, player, high, season, team, coll...","['matt', 'youmans', 'reviewjournal', 'plain', ..."
196,9428,1,0.5855,"school, year, player, high, season, team, coll...","['jordan', 'call', 'career', 'certainly', 'las..."
197,76467,1,0.5855,"school, year, player, high, season, team, coll...","['long', 'time', 'since', 'temple', 'experienc..."
198,62090,1,0.585,"school, year, player, high, season, team, coll...","['former', 'bull', 'assistant', 'coach', 'tex'..."



--- Topic 2 ---


...

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
1700,63571,17,0.9003,"point, scored, second, lead, first, state, reb...","['check', 'ncaa', 'tournament', 'story', 'phot..."
1701,63969,17,0.8952,"point, scored, second, lead, first, state, reb...","['one', 'day', 'cal', 'state', 'northridge', '..."
1702,28279,17,0.8883,"point, scored, second, lead, first, state, reb...","['eric', 'musselman', 'fired', 'wednesday', 't..."
1703,83052,17,0.8809,"point, scored, second, lead, first, state, reb...","['hoover', 'ala', 'hoover', 'ala', 'indiana', ..."
1704,72243,17,0.8807,"point, scored, second, lead, first, state, reb...","['question', 'arose', 'recently', 'regarding',..."
...,...,...,...,...,...
1795,50477,17,0.7909,"point, scored, second, lead, first, state, reb...","['ojai', 'day', 'university', 'nevada', 'la', ..."
1796,89638,17,0.7909,"point, scored, second, lead, first, state, reb...","['maine', 'may', 'locked', 'colonial', 'athlet..."
1797,18321,17,0.7908,"point, scored, second, lead, first, state, reb...","['rugby', 'u', 'team', 'face', 'tough', 'opene..."
1798,71159,17,0.7908,"point, scored, second, lead, first, state, reb...","['washing', 'dish', 'seafood', 'restaurant', '..."



--- Topic 18 ---


,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
1800,29156,18,0.9533,"sport, game, basketball, network, college, fan...","['last', 'row', 'bold', 'name', 'mitch', 'albo..."
1801,79729,18,0.9372,"sport, game, basketball, network, college, fan...","['houston', 'somewhere', 'connecticut', 'fall'..."
1802,31218,18,0.9198,"sport, game, basketball, network, college, fan...","['tennessee', '59', 'rutgers', '49', 'througho..."
1803,5039,18,0.9133,"sport, game, basketball, network, college, fan...","['university', 'park', 'pa', 'semifinal', 'thu..."
1804,33437,18,0.9122,"sport, game, basketball, network, college, fan...","['former', 'baylor', 'basketball', 'player', '..."
...,...,...,...,...,...
1895,88065,18,0.7518,"sport, game, basketball, network, college, fan...","['ben', 'howland', 'fault', 'put', 'blame', 'u..."
1896,100886,18,0.7517,"sport, game, basketball, network, college, fan...","['junior', 'sam', 'mattis', 'penn', 'uncorked'..."
1897,87797,18,0.7491,"sport, game, basketball, network, college, fan...","['let', 'sink', 'every', 'position', 'player',..."
1898,73009,18,0.7488,"sport, game, basketball, network, college, fan...","['connecticut', 'coach', 'jim', 'calhoun', 'me..."



--- Topic 19 ---


,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
1900,38552,19,0.8077,"ucla, usc, bruin, trojan, oregon, los, angeles...","['chris', 'kenny', 'drove', 'length', 'court',..."
1901,46055,19,0.7235,"ucla, usc, bruin, trojan, oregon, los, angeles...","['top', 'seed', 'awarded', 'florida', 'ohio', ..."
1902,9233,19,0.7065,"ucla, usc, bruin, trojan, oregon, los, angeles...","['ncaa', 'tournament', '2', '1', '2', 'decade'..."
1903,28722,19,0.6714,"ucla, usc, bruin, trojan, oregon, los, angeles...","['college', 'basketball', 'look', 'stage', 'tw..."
1904,31712,19,0.5978,"ucla, usc, bruin, trojan, oregon, los, angeles...","['hofstra', 'football', 'coach', 'joe', 'gardi..."
...,...,...,...,...,...
1995,89067,19,0.3841,"ucla, usc, bruin, trojan, oregon, los, angeles...","['final', 'high', 'school', 'girl', 'basketbal..."
1996,63814,19,0.3832,"ucla, usc, bruin, trojan, oregon, los, angeles...","['ucla', 'open', 'second', 'football', 'season..."
1997,117053,19,0.3831,"ucla, usc, bruin, trojan, oregon, los, angeles...","['malibu', 'ocean', 'become', 'bluer', 'since'..."
1998,115506,19,0.383,"ucla, usc, bruin, trojan, oregon, los, angeles...","['columbus', 'ohio', 'geno', 'auriemma', 'warn..."


In [10]:
# Define the path to save the Excel file in your Google Drive
excel_file_path = '/content/drive/MyDrive/Colab Notebooks/alex/top_100_articles/top_100_documents_per_topic_for_20_topics.xlsx'

# Save the DataFrame to an Excel file
try:
    top_100_documents_per_topic.to_excel(excel_file_path, index=False)
    print(f"Successfully saved the top 100 documents per topic to: {excel_file_path}")
except Exception as e:
    print(f"An error occurred while saving the Excel file: {e}")

Successfully saved the top 100 documents per topic to: /content/drive/MyDrive/Colab Notebooks/alex/top_100_articles/top_100_documents_per_topic_for_20_topics.xlsx


##### **© 2024–2025 MD Rafsun Sheikh**
##### **Licensed under the Apache License, Version 2.0.**